In [21]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datetime

In [31]:
df = pd.read_csv('data.csv', index_col=0, parse_dates=True, infer_datetime_format=True, low_memory=False)

In [35]:
def fromNsTimestampToDatetime(timestamp):
    return datetime.datetime.fromtimestamp(timestamp / 1e6)

df['date'] = df['timestamp'].apply(fromNsTimestampToDatetime)

In [36]:
df.head()

,symbol,timestamp,local_timestamp,asks[0].price,asks[0].amount,bids[0].price,bids[0].amount,asks[1].price,asks[1].amount,bids[1].price,...,bids[2].amount,asks[3].price,asks[3].amount,bids[3].price,bids[3].amount,asks[4].price,asks[4].amount,bids[4].price,bids[4].amount,date
exchange,,,,,,,,,,,,,,,,,,,,,
ftx,BTC-PERP,1619481600260874,1619481600280633,54059,5.1795,54058,0.0600,54060,0.1665,54051,...,0.0935,54067,1.1252,54047,0.3138,54068,0.3668,54046,0.1487,2021-04-27 02:00:00.260874
ftx,BTC-PERP,1619481600335591,1619481600358205,54059,5.1795,54058,0.0600,54060,0.1665,54051,...,0.0935,54067,1.1252,54047,0.3138,54068,0.3668,54046,0.1487,2021-04-27 02:00:00.335591
ftx,BTC-PERP,1619481600416152,1619481600434715,54059,5.1795,54051,0.0500,54060,0.1665,54050,...,0.3138,54067,1.1252,54046,0.1487,54068,0.3668,54045,3.1385,2021-04-27 02:00:00.416152
ftx,BTC-PERP,1619481600689763,1619481600730319,54051,0.1200,54045,0.1306,54052,0.3440,54044,...,0.2708,54058,0.8700,54041,0.0944,54059,6.0314,54040,2.0926,2021-04-27 02:00:00.689763
ftx,BTC-PERP,1619481600935299,1619481600988681,54048,0.0600,54040,0.0100,54049,27.8292,54039,...,2.1396,54052,25.2858,54033,0.0925,54058,0.8200,54032,0.2126,2021-04-27 02:00:00.935299


In [74]:
# find col names for bid,asks, and trades
ask_price_cols = df.columns[(df.columns.str.startswith('ask')) & (df.columns.str.endswith('price'))]
ask_amount_cols = df.columns[(df.columns.str.startswith('ask')) & (df.columns.str.endswith('amount'))]
bid_price_cols = df.columns[(df.columns.str.startswith('bid')) & (df.columns.str.endswith('price'))]
bid_amount_cols = df.columns[(df.columns.str.startswith('bid')) & (df.columns.str.endswith('amount'))]

bid_amount_cols

Index(['bids[0].amount', 'bids[1].amount', 'bids[2].amount', 'bids[3].amount',
       'bids[4].amount'],
      dtype='object')

In [75]:
def RowToOrders(row):
    limit_orders = []

    for x,y,z in zip(row[ask_price_cols].values, row[ask_amount_cols].values, ['A']*5):
        limit_orders.append({'price': x, 'amount': y, 'side': z})
    
    for x,y,z in zip(row[bid_price_cols].values, row[bid_amount_cols].values, ['B']*5):
        limit_orders.append({'price': x, 'amount': y, 'side': z})

    return limit_orders

In [140]:
def RowToOrders(row):
    limit_price, amount, side = [], [], []

    for x,y,z in zip(row[ask_price_cols].values, row[ask_amount_cols].values, ['A']*5):
        limit_price.append(x)
        amount.append(y)
        side.append(z)
    
    for x,y,z in zip(row[bid_price_cols].values, row[bid_amount_cols].values, ['B']*5):
        limit_price.append(x)
        amount.append(y)
        side.append(z)

    return [limit_price, amount, side, row['date']]

In [141]:
ob = RowToOrders(df.iloc[1])
ob

[[54059, 54060, 54063, 54067, 54068, 54058, 54051, 54050, 54047, 54046],
 [5.1795, 0.1665, 0.1196, 1.1252, 0.3668, 0.06, 0.1, 0.0935, 0.3138, 0.1487],
 ['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B'],
 Timestamp('2021-04-27 02:00:00.335591')]

In [161]:
colors = {'A':'firebrick',
          'B':'green'}

def ListOfDictToPlot(row):
    fig = go.Figure()
    fig.add_trace(go.Bar(x=row[0], y=row[1],marker_color=[colors[i] for i in ob[2]]))
    fig.update_xaxes(range=[54000,54100])
    fig.update_layout(
        xaxis_tickprefix = '$',
        xaxis_tickformat =',',
        xaxis_title = 'Limit Price',
        yaxis_title = 'Amount',
        title = str(row[3])
        )
    fig.show()

In [162]:
for i in range(1):
    ListOfDictToPlot(RowToOrders(df.iloc[i]))